- get chatgpt to accept prompts
- get chatgpt to output prompts

In [ ]:
!pip install openai

In [ ]:
import os
import openai
from google.colab import userdata
openai.api_key = userdata.get(API_KEY)

In [ ]:
import pandas as pd

In [ ]:
#Initial call to GPT to understand the trials
init_sys_call = [
    {"role": "system",
    "content": """Your task is to give judgement about where you were based on the time we give. You will give 4 alternative options. You MUST select one.

                  Example:

                  Presented:
                  1 at 6PM 11/08/2019
                  2 at 7PM 12/08/2019
                  ...

                  Question:
                  Where were you at 6PM 11/09/2019:
                  A. 2
                  B. 3
                  C. 1
                  D. 30

                  Response:
                  C

                  Where were you at 7PM 12/09/2019:
                  A. 2
                  B. 3
                  C. 9
                  D. 30

                  Response:
                  A
                  ..."""
              }]

In [ ]:
events = pd.read_csv('events.csv')
experiments = pd.read_csv('experiment.csv')

# Base Case Events - 1st User's ID, StartDateTime UTC, GPS Cluster Original
USER_ID = "ap-northeast-1:2e3dfd5b-19ff-4b99-abe5-bbee4d46749f"
events_1 = events[['ID', 'StartDateTime Local', 'GPS Cluster Original']][events['USER ID']==USER_ID]


# Base Case Experiment - 1st ID, Target, A, B, C, D
experiments_1 = experiments[['ID', 'Target', 'A', 'B', 'C', 'D']][experiments['USER ID']==USER_ID]

# offsets
events_1_offset = events_1.index.min()
experiments_1_offset = experiments_1.index.min()
experiments_1


,ID,Target,A,B,C,D
461,461,A,2633,2557,2695,2608
462,462,A,2678,2513,2680,2695
463,463,B,2749,2752,2724,2796
464,464,C,2583,2678,2774,2508
465,465,C,2616,2658,2677,2698
...,...,...,...,...,...,...
548,548,A,2540,2677,2610,2682
549,549,C,2537,2581,2638,2774
550,550,C,2660,2680,2612,2607
551,551,C,2587,2608,2823,2658


In [ ]:
# Let GPT know where it was
import re
location_data = []

for i in range(len(events_1)):#len(events_1)):
  datetime = events_1.loc[i+events_1_offset, "StartDateTime Local"] ## NEED TO ADJUST INDEX
  date_pattern = r"(\d{4}-\d{2}-\d{2})T"
  time_pattern = r"\d{4}-\d{2}-\d{2}T(\d{2}:\d{2}:\d{2})Z"
  date = re.search(date_pattern, datetime).group(1)
  time = re.search(time_pattern, datetime).group(1)

  gps_cluster = events_1.loc[i+events_1_offset, "GPS Cluster Original"] ## NEED TO ADJUST INDEX
  location_data.append({
      "role": "user",
      "content": f"{gps_cluster} at {time} {date}"
  })
  # 2019-08-11T06:00:00Z
  #  print(events_1.loc[i, "ID"], events_1.loc[i, "GPS Cluster Original"])

location_data

[{'role': 'user', 'content': '0 at 00:00:00 2019-07-29'},
 {'role': 'user', 'content': '0 at 01:00:00 2019-07-29'},
 {'role': 'user', 'content': '0 at 02:00:00 2019-07-29'},
 {'role': 'user', 'content': '0 at 03:00:00 2019-07-29'},
 {'role': 'user', 'content': '0 at 04:00:00 2019-07-29'},
 {'role': 'user', 'content': '0 at 05:00:00 2019-07-29'},
 {'role': 'user', 'content': '0 at 06:00:00 2019-07-29'},
 {'role': 'user', 'content': '0 at 07:00:00 2019-07-29'},
 {'role': 'user', 'content': '0 at 08:00:00 2019-07-29'},
 {'role': 'user', 'content': '1 at 09:00:00 2019-07-29'},
 {'role': 'user', 'content': '1 at 10:00:00 2019-07-29'},
 {'role': 'user', 'content': '1 at 11:00:00 2019-07-29'},
 {'role': 'user', 'content': '2 at 12:00:00 2019-07-29'},
 {'role': 'user', 'content': '3 at 13:00:00 2019-07-29'},
 {'role': 'user', 'content': '2 at 14:00:00 2019-07-29'},
 {'role': 'user', 'content': '3 at 15:00:00 2019-07-29'},
 {'role': 'user', 'content': '3 at 16:00:00 2019-07-29'},
 {'role': 'use

In [ ]:
# Form Questions for GPT
question_data = []

for i in range(len(experiments_1)):#len(experiments_1)):
  question = ""
  response_id = experiments_1.loc[i+experiments_1_offset, "Target"] ## NEED TO ADJUST INDEX
  correct_event_id = experiments_1.loc[i+experiments_1_offset, response_id] ## NEED TO ADJUST INDEX

  # Cluster for A, B, C, D respectively
  cluster_question = []
  for alpha in ['A', 'B', 'C', 'D']:
    alpha_response_id = experiments_1.loc[i+experiments_1_offset, alpha] ## NEED TO ADJUST INDEX
    cluster_question.append(events_1.loc[alpha_response_id, "GPS Cluster Original"])

  datetime = events_1.loc[correct_event_id, "StartDateTime Local"]
  date_pattern = r"(\d{4}-\d{2}-\d{2})T"
  time_pattern = r"\d{4}-\d{2}-\d{2}T(\d{2}:\d{2}:\d{2})Z"
  date = re.search(date_pattern, datetime).group(1)
  time = re.search(time_pattern, datetime).group(1)

  question += f"Where were you at {time} {date}:\n"
  question += f"A. {cluster_question[0]}\n"
  question += f"B. {cluster_question[1]}\n"
  question += f"C. {cluster_question[2]}\n"
  question += f"D. {cluster_question[3]}"

  question_data.append({
      "role": "user",
      "content": question
  })

question_data

[{'role': 'user',
  'content': 'Where were you at 15:00:00 2019-08-10:\nA. 0\nB. 13\nC. 17\nD. 3'},
 {'role': 'user',
  'content': 'Where were you at 15:00:00 2019-08-12:\nA. 3\nB. 0\nC. 14\nD. 17'},
 {'role': 'user',
  'content': 'Where were you at 17:00:00 2019-08-15:\nA. 7\nB. 3\nC. 7\nD. 0'},
 {'role': 'user',
  'content': 'Where were you at 15:00:00 2019-08-16:\nA. 2\nB. 3\nC. 19\nD. 0'},
 {'role': 'user',
  'content': 'Where were you at 14:00:00 2019-08-12:\nA. 0\nB. 0\nC. 3\nD. 6'},
 {'role': 'user',
  'content': 'Where were you at 13:00:00 2019-08-10:\nA. 2\nB. 0\nC. 19\nD. 3'},
 {'role': 'user',
  'content': 'Where were you at 12:00:00 2019-08-05:\nA. 7\nB. 6\nC. 0\nD. 2'},
 {'role': 'user',
  'content': 'Where were you at 13:00:00 2019-08-09:\nA. 19\nB. 3\nC. 0\nD. 2'},
 {'role': 'user',
  'content': 'Where were you at 10:00:00 2019-08-13:\nA. 3\nB. 0\nC. 6\nD. 2'},
 {'role': 'user',
  'content': 'Where were you at 14:00:00 2019-08-11:\nA. 0\nB. 2\nC. 3\nD. 0'},
 {'role': 'us

In [ ]:
output_msg = []
message = {}

for i in range(len(experiments_1)):
  message = init_sys_call + location_data

  if (i==0):
    message.append(question_data[0])
  else:
    for j in range(i+1):
      message.append(question_data[j])
      if (j<i):
        message.append({'role': 'assistant', 'content': output_msg[j]})

  # output_msg.append('A')
  # print(message)
  response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    messages=message
  )

  # print(response)

  if (response['choices'][0]['finish_reason'] == "length"):
    print("ERROR: Too Long")
    break

  output_msg.append(response['choices'][0]['message']['content'])

output_msg



Timeout: ignored

In [ ]:
len(output_msg)

27

In [ ]:
response

<OpenAIObject chat.completion id=chatcmpl-8ABJAQupInduw6mDMTI5bUmAkALhZ at 0x7bcc5275d8a0> JSON: {
  "id": "chatcmpl-8ABJAQupInduw6mDMTI5bUmAkALhZ",
  "object": "chat.completion",
  "created": 1697437440,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "B"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 10738,
    "completion_tokens": 1,
    "total_tokens": 10739
  }
}

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(output_msg, columns=["Response"])
df.to_csv('/content/data/responseFULLparticipant7.csv', index=True)